### import 및 설정값

In [1]:
import os
os.environ["WANDB_DISABLED"] = "True"

import pandas as pd
import numpy as np
import cv2
import shutil
import yaml
import warnings
warnings.filterwarnings("ignore")

from ultralytics import YOLO
from glob import glob
from tqdm import tqdm
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

In [6]:
SEED = 42
BATCH_SIZE = 8
MODEL = "dacon"

In [3]:
def get_test_image_paths(test_image_paths):    
    for i in range(0, len(test_image_paths), BATCH_SIZE):
        yield test_image_paths[i:i+BATCH_SIZE]

### 추론

In [4]:
model = YOLO("dacon/train/weights/best.pt")
test_image_paths = glob("data/yolo/test/*.png")
for i, image in tqdm(enumerate(get_test_image_paths(test_image_paths)), total=int(len(test_image_paths)/BATCH_SIZE)):
    model.predict(image, imgsz=(1024, 1024), iou=0.2, conf=0.5, save_conf=True, save=False, save_txt=True, project=f"{MODEL}", name="predict",
                  exist_ok=True, device=0, augment=True, verbose=False)
    if i % 5 == 0:
        clear_output(wait=True)

 99%|█████████▉| 421/425 [06:22<00:03,  1.10it/s]Results saved to v2/predict
3373 labels saved to v2/predict/labels
 99%|█████████▉| 422/425 [06:23<00:02,  1.10it/s]Results saved to v2/predict
3381 labels saved to v2/predict/labels
100%|█████████▉| 423/425 [06:24<00:01,  1.10it/s]Results saved to v2/predict
3389 labels saved to v2/predict/labels
100%|█████████▉| 424/425 [06:25<00:00,  1.10it/s]Results saved to v2/predict
3397 labels saved to v2/predict/labels
100%|██████████| 425/425 [06:26<00:00,  1.10it/s]


### YOLO 라벨링 -> LabelMe 라벨링 형식으로 변환

In [5]:
def yolo_to_labelme(line, image_width, image_height, txt_file_name):    
    file_name = txt_file_name.split("/")[-1].replace(".txt", ".png")
    class_id, x, y, width, height, confidence = [float(temp) for temp in line.split()]
    
    x_min = int((x - width / 2) * image_width)
    x_max = int((x + width / 2) * image_width)
    y_min = int((y - height / 2) * image_height)
    y_max = int((y + height / 2) * image_height)
    
    return file_name, int(class_id), confidence, x_min, y_min, x_max, y_min, x_max, y_max, x_min, y_max

In [8]:
infer_txts = glob(f"{MODEL}/predict/labels/*.txt")

results = []
for infer_txt in tqdm(infer_txts):
    base_file_name = infer_txt.split("/")[-1].split(".")[0]
    imgage_height, imgage_width = cv2.imread(f"data/yolo/test/{base_file_name}.png").shape[:2]        
    with open(infer_txt, "r") as reader:        
        lines = reader.readlines()        
        for line in lines:
            results.append(yolo_to_labelme(line, imgage_width, imgage_height, infer_txt))

df_submission = pd.DataFrame(data=results, columns=["file_name", "class_id", "confidence", "point1_x", "point1_y", "point2_x", "point2_y", "point3_x", "point3_y", "point4_x", "point4_y"])
df_submission.to_csv(f"results/{MODEL}.csv", index=False)

100%|██████████| 3397/3397 [02:03<00:00, 27.46it/s]
